In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
import keras

from settings import *

from utils.general import clean_session, plot_model, split_dataset, scale_dataset, shape_dataset, print_dataset_info
from utils.models.ocr import save_model

In [ ]:
import Datasets.OCR.CNN_Letters.loader as CNN_Letters
import Datasets.OCR.manual.loader as manual
import Datasets.OCR.EnglishFnt.loader as EnglishFnt

In [ ]:
# clean_session()

In [ ]:
def create_model():
    model = Sequential()

    # CNN
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=INPUT_SHAPE))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))

    # Pooling
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))

    # Flattening before feeding to FCNN
    model.add(Flatten())

    # FCNN
    model.add(Dense(64, activation='relu'))
    # use softmax since we are doing classification (1 node for each possible output)
    model.add(Dense(len(COMBINED_CHARS), activation='softmax'))

    model.summary()

    return model


In [ ]:
def train_model(model, learning_rate, batch_size, epochs, patience):
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True, verbose=1)]

    CX_data, CY_data = CNN_Letters.load_dataset()
    EX_data, EY_data = EnglishFnt.load_dataset(include_fonts=ENGLISH_FNT_FONTS)
    MX_data, MY_data = manual.load_dataset()
    X_data = np.concatenate((CX_data, MX_data, EX_data))
    Y_data = np.concatenate((CY_data, MY_data, EY_data))

    X_train, X_test, Y_train, Y_test = split_dataset(X_data, Y_data, test_percent=TEST_PERCENT, random_state=RANDOM_STATE)
    X_train, X_test = shape_dataset(X_train, X_test, input_shape=INPUT_SHAPE)
    X_train, X_test = scale_dataset(X_train, X_test)

    print_dataset_info(X_data, X_train, Y_train, X_test, Y_test)

    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_split=VALIDATION_PERCENT, callbacks=callbacks)

    return history

In [ ]:
model = create_model()
history = train_model(model, learning_rate=0.001, batch_size=16, epochs=50, patience=5)
plot_model(history)
save_model(model, 3)